In [1]:
import numpy as np
import random

In [107]:
card_values = {'2','3','4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A'}
card_suits = {'Hearts', 'Diamonds', 'Spades', 'Clubs'}
class Card:
    """Card class for game - units to make up deck"""
    def __init__(self, value, suit):
        self.value = value
        self.suit = suit
        
    def __repr__(self):
        return f'{self.value} of {self.suit}'

    
class Deck:
    """Deck class - made up of cards, to deal to players in game"""
    def __init__(self, cards):
        self.cards = cards
        self.deck_size = len(cards)
        self.dealt = [] # keeps track of dealt cards
        
    def __repr__(self):
        return "\n".join([f"{card}" for card in self.cards])
    
    def __getitem__(self, indices):
        return self.cards[indices]
    
    def shuffle(self, times = 3):
        for t in range(0, times):
            np.random.shuffle(self.cards)
    
    def deal(self, n):
        assert n <= len(self.cards), "Cannot deal more cards than available"
        to_deal = self.cards[0:n]
        self.dealt += to_deal # keep track of dealt cards
        self.cards = self.cards[n:] # remove cards from deck
        return to_deal
    
    def reclaimCards(self):
        self.cards += self.dealt
        self.dealt = []      

class Game:
    """class for the running of the game"""
    
    def __init__(self, deck, players):
        self.deck = deck
        self.players = players
        
        
    def deal(self):
        """initial dealing of cards"""
        for player in self.players:
            player.getHand(self.deck.deal(2))
            
    def play(self):
        self.deck.shuffle(5) # shuffle the deck
        self.deal() # make sure each player has cards
        # make sure players know dealer showing
        for player in self.players:
            hit = True
            while not player.bust and hit:

                hit = player.hitOrStay()
                print('BEFORE:',player.hand, player.score, '\n-------------\n')
                if hit:
                    player.addCard(self.deck.deal(1))
                print('\n------------\n', 'bust?',player.bust, 'hit?',hit)
                print(player.hand, player.score, '\n-------------\n')
        print('Game Over! Reclaiming cards')
        self.deck.reclaimCards()  
        
        
        
class Player:
    
    def __init__(self):
        self.hand = None
        self.score = 0
        self.bust = False
    
    def hitOrStay(self):
        return True if np.random.uniform() <= 0.9 else False
    
    def getHand(self, cards):
        self.hand = cards
        self.calcScore()
    
    def addCard(self, new):
        self.hand += new
        self.calcScore()
        
    def calcScore(self):
        temp_score = 0
        non_aces = [card for card in self.hand if card.value != 'A']
        aces = [card for card in self.hand if card.value == 'A']
        
        # check that there actually are cards
        num_non_aces = len(non_aces)
        num_aces = len(aces)
        
        if num_non_aces > 0:
            for card in non_aces:
                if card.value.isdigit():
                    temp_score += int(card.value)
                elif card.value in {'K', 'Q', 'J'}:
                    temp_score += 10
                
        if num_aces > 0:
            # checks for rare cases on multiple aces
            if num_aces == 1:
                if temp_score + 11 <= 21:
                    temp_score += 11
                else:
                    temp_score += 1
            else:
                if temp_score + 11 + 1 * (num_aces - 1) <= 21:
                    temp_score += 11 + 1 * (num_aces - 1)
                else:
                    temp_score += 1 * num_aces
        
        self.score = temp_score
        
        if self.score > 21:
            self.bust = True

            
            
            

In [108]:
a_deck_of_cards = [Card(value, suit) for value in card_values for suit in card_suits]

aDeck = Deck(a_deck_of_cards)

aplayer = Player()

aGame = Game(aDeck, [aplayer])

aGame.play()


BEFORE: [3 of Spades, A of Hearts] 14 
-------------


------------
 bust? False hit? True
[3 of Spades, A of Hearts, A of Spades] 15 
-------------

BEFORE: [3 of Spades, A of Hearts, A of Spades] 15 
-------------


------------
 bust? False hit? True
[3 of Spades, A of Hearts, A of Spades, K of Hearts] 15 
-------------

BEFORE: [3 of Spades, A of Hearts, A of Spades, K of Hearts] 15 
-------------


------------
 bust? True hit? True
[3 of Spades, A of Hearts, A of Spades, K of Hearts, 7 of Clubs] 22 
-------------

Game Over! Reclaiming cards


In [109]:
all_aces = [Card('3', 'Spades'), Card('A', 'Hearts'), Card('A', 'Spades'), Card('K', 'Hearts')]
aplayer = Player()
aplayer.getHand(all_aces)
aplayer.calcScore()
print(aplayer.score, aplayer.bust)

15 False


In [88]:
print(aplayer.hand)

None


In [39]:
for i in range(0, 26):
    print(i)
    k = aDeck.deal(2)
    print(aDeck[0:3], k)

0
[5 of Spades, 5 of Diamonds, 8 of Hearts] [5 of Hearts, 5 of Clubs]
1
[8 of Hearts, 8 of Clubs, 8 of Spades] [5 of Spades, 5 of Diamonds]
2
[8 of Spades, 8 of Diamonds, 3 of Hearts] [8 of Hearts, 8 of Clubs]
3
[3 of Hearts, 3 of Clubs, 3 of Spades] [8 of Spades, 8 of Diamonds]
4
[3 of Spades, 3 of Diamonds, Q of Hearts] [3 of Hearts, 3 of Clubs]
5
[Q of Hearts, Q of Clubs, Q of Spades] [3 of Spades, 3 of Diamonds]
6
[Q of Spades, Q of Diamonds, 6 of Hearts] [Q of Hearts, Q of Clubs]
7
[6 of Hearts, 6 of Clubs, 6 of Spades] [Q of Spades, Q of Diamonds]
8
[6 of Spades, 6 of Diamonds, 7 of Hearts] [6 of Hearts, 6 of Clubs]
9
[7 of Hearts, 7 of Clubs, 7 of Spades] [6 of Spades, 6 of Diamonds]
10
[7 of Spades, 7 of Diamonds, 9 of Hearts] [7 of Hearts, 7 of Clubs]
11
[9 of Hearts, 9 of Clubs, 9 of Spades] [7 of Spades, 7 of Diamonds]
12
[9 of Spades, 9 of Diamonds, A of Hearts] [9 of Hearts, 9 of Clubs]
13
[A of Hearts, A of Clubs, A of Spades] [9 of Spades, 9 of Diamonds]
14
[A of Spades,

In [37]:
aDeck.reclaimCards()

In [38]:
aDeck[0:3]

[5 of Hearts, 5 of Clubs, 5 of Spades]